## 中央競馬の情報をスクレイピングで抽出するプログラム

In [25]:
import os
os.chdir(r"C:\Users\shira\デスクトップ\programming\python\horse_racing")
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import service as fs


In [26]:
chrome_service = fs.Service(executable_path=r"..\chromedriver.exe")
browser = webdriver.Chrome(service=chrome_service)
browser.get("https://db.netkeiba.com/race/202003010302")

html = browser.page_source.encode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

### レース情報抽出

### 馬情報の抽出

In [27]:
soup_span = soup.find_all("span")
N = int((len(soup_span)-9)/3) + 1 #馬の数

soup_txt_l = soup.find_all(class_="txt_l")
soup_txt_r=soup.find_all(class_="txt_r")
soup_nowrap = soup.find_all("td",nowrap="nowrap",class_=None)
soup_txt_c = soup.find_all("td",nowrap="nowrap",class_="txt_c")
name, jockey, horse_num, runtime, odds, pas, weight, sex_and_old, handicap, last, popular= [], [], [], [], [], [], [], [], [], [], []

for i in range(N):
    name.append(soup_txt_l[4*i].contents[1].contents[0])
    jockey.append(soup_txt_l[4*i + 1].contents[1].contents[0])
    horse_num.append(soup_txt_r[5*i + 1].contents[0])
    runtime.append(soup_txt_r[5*i + 2].contents[0])    
    odds.append(soup_txt_r[5*i + 3].contents[0])
    pas.append(soup_nowrap[3*i].contents[0])
    weight.append(soup_nowrap[3*i + 1].contents[0])
    sex_and_old.append(soup_txt_c[5*i].contents[0])
    handicap.append(soup_txt_c[5*i + 1].contents[0])
    last.append(soup_span[3*i + 6].contents[0])
    popular.append(soup_span[3*i + 7].contents[0])



horse_info = [name, jockey, horse_num, runtime, odds, pas, weight, sex_and_old, handicap, last, popular]
df_horse_info = pd.DataFrame(horse_info, index=["馬名", "騎手", "馬番", "タイム", "オッズ", "通過", "体重", "性齢", "斤量", "上がり", "人気"])
display(df_horse_info)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
馬名,リヴェール,シエルヴァ,メイリバティ,フローラルドレス,レオハイセンス,ナンヨーショウエイ,ゴールドシンガー,プラチナビューティ,キラビヤカ,ブロッケンリング,シンキングムーン,メイショウウタヒメ,ババルダージュ,ノーブルルビー,ジョーパティオ,エリーフォース
騎手,西村淳也,原優介,斎藤新,勝浦正樹,黛弘人,藤田菜七,杉原誠人,丸田恭介,中谷雄太,団野大成,木幡初也,国分優作,鮫島克駿,亀田温心,城戸義政,菅原明良
馬番,9,7,4,12,15,8,13,5,2,16,3,11,6,1,10,14
タイム,1:10.6,1:10.6,1:10.7,1:10.9,1:10.9,1:11.2,1:11.3,1:11.4,1:11.4,1:11.6,1:11.8,1:12.0,1:12.3,1:12.3,1:12.5,1:14.8
オッズ,5.4,11.3,2.6,7.7,64.3,5.3,307.4,34.3,129.1,37.6,14.3,198.1,37.7,11.9,134.6,390.1
通過,2-2,3-4,1-1,3-3,7-4,11-10,6-6,12-12,9-9,14-13,7-6,13-13,9-10,15-15,3-6,16-16
体重,436(+2),448(+2),442(-2),422(-6),438(-6),394(-2),470(+6),420(-16),416(+12),422(+6),444(+6),420(+10),406(+14),480(0),442(-10),440(-2)
性齢,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3,牝3
斤量,53,51,53,54,54,52,54,54,54,52,54,54,54,51,54,52
上がり,36.3,36.1,36.4,36.5,36.2,36.1,36.7,36.1,36.5,36.0,37.1,36.6,37.3,36.0,38.0,38.2


### 払い戻し情報の抽出

In [28]:

payback = soup.find_all("table", summary='払い戻し')
payback_info=[]

def append_payback1(input_soup): #複勝とワイド以外
    tmp_list = []
    tmp_list.append(input_soup.contents[3].contents[0])
    tmp_list.append(input_soup.contents[5].contents[0])
    payback_info.append(tmp_list)


def append_payback2(input_soup): #複勝とワイド
    tmp_list = []
    for i in range(3):
        tmp_list.append(input_soup.contents[3].contents[2*i])
        tmp_list.append(input_soup.contents[5].contents[2*i])
    payback_info.append(tmp_list)


for i in range(2):
    for j in range(4):
        if ((i==0) and (j==1)) or ((i==1) and (j==0)):
            append_payback2(payback[i].contents[1].contents[2*j])
        else:
            append_payback1(payback[i].contents[1].contents[2*j])  


df_payback_info = pd.DataFrame(payback_info, index=["単勝", "複勝", "枠連", "馬連", "ワイド", "馬単", "三連複", "三連単"])
display(df_payback_info)

,0,1,2,3,4,5
単勝,9,540,None,None,None,None
複勝,9,160,7,260,4,140
枠連,4 - 5,"1,160",None,None,None,None
馬連,7 - 9,"2,800",None,None,None,None
ワイド,7 - 9,640,4 - 9,360,4 - 7,320
馬単,9 → 7,"6,160",None,None,None,None
三連複,4 - 7 - 9,"1,980",None,None,None,None
三連単,9 → 7 → 4,"20,780",None,None,None,None
